# Proyecto: Clasificación de Productos con Aprendizaje por Refuerzo (Q-Learning)

Este proyecto simula una cinta transportadora como las que se encuentran en fábricas o plantas industriales.  
Un agente inteligente aprende a tomar decisiones por sí mismo sobre:

- Cuándo retirar un producto defectuoso.
- Cuándo dejar pasar un producto en buen estado.

Para esto se utiliza **Aprendizaje por Refuerzo**, específicamente el algoritmo **Q-Learning**, donde el agente aprende mediante recompensas.


# pregunta de final  


Modifica el código para que se considere un tercer producto que se desplace a otro contenedor, el cual corresponde a un producto que se puede corregirlo posteriormente



In [1]:
import numpy as np
import random
import time
import os
import matplotlib.pyplot as plt

## Entorno del Proyecto

El entorno representa una cinta transportadora con productos:  
(estados)
- 0 → producto bueno
- 1 → producto defectuoso

Acciones del agente:
- 0 = dejar pasar
- 1 = retirar

Recompensas:
- +10 si toma la decisión correcta
- -5 si se equivoca


ahora estoy agregando uno que es  
0 -> bueno  
1 -> reparable  
2 -> malo

In [2]:
class CintaTransportadoraEnv:

#agregaremoos otro para el producto que se pude reparar , con una probabilidad del 20%s
    def __init__(self, prob_malo=0.3,prob_reparable=0.2, semilla=None):
        self.prob_malo = prob_malo
        self.prob_reparable= prob_reparable
        self.rng = random.Random(semilla)
        self.producto_actual = None

    def reiniciar(self):
        
        rando= self.rng.random()
        
        #vemos la probabilidad de que sea bueno
        
        prob_bueno= 1.0 - self.prob_reparable - self.prob_malo
        
        if rando < prob_bueno:
            
            self.producto_actual = 0 
        elif rando < prob_bueno + self.prob_reparable:
            
            self.producto_actual = 1 
        else:
            self.producto_actual = 2  
        
        return self.producto_actual
        
        #self.producto_actual = 1 if self.rng.random() < self.prob_malo else 0
        #return self.producto_actual

    def paso(self, accion):
        
        
        
        #0  dejar pasar
        #1  enviar a reparación
        #2  desechar
     
   
        prod = self.producto_actual
        
        if accion == 0:  
            if prod == 0:   
                recompensa = 5
            elif prod == 1:  
                recompensa = -8
            else:            
                recompensa = -15
                
        elif accion == 1:  
            if prod == 0:    
                recompensa = -3
            elif prod == 1:  
                recompensa = 8
            else:            
                recompensa = -5
                
        else:  
            if prod == 0:   
                recompensa = -5
            elif prod == 1:  
                recompensa = -2
            else:            
                recompensa = 10
            
        
        
        
        
        #if accion == 1:  # retirar
        #    recompensa = 10.0 if prod == 1 else -5.0
        #else:            # dejar pasar
        #    recompensa = 5.0 if prod == 0 else -10.0

        #self.producto_actual = 1 if self.rng.random() < self.prob_malo else 0
        
        
        
        
        
        #reiniciamos todo y generamos el producto siguiente
        self.producto_actual = self.reiniciar()
        terminado = False
        info = {"producto": prod}
        return self.producto_actual, recompensa, terminado, info


## Algoritmo Q-Learning

El agente aprende una tabla Q con valores para cada acción.  
Actualiza la tabla con la fórmula:

Q(s,a) = Q(s,a) + α [ r + γ max(Q(s’, a’)) − Q(s,a) ]

Con esto aprende qué hacer en cada estado.


In [ ]:
def entrenamiento(episodios, alpha, gamma,epsilon, decay_epsilon, min_epsilon,prob_reparable ,prob_malo, semilla, guardar_ruta):
    env = CintaTransportadoraEnv(prob_reparable=prob_reparable,prob_malo=prob_malo,semilla=semilla)
   #ahora tendremos 3 acciones y 3 estados eso es lo qye cambie
    n_estados=3
    n_acciones=3
    
    #iniciamos en ceros
    Q = np.zeros((n_estados,n_acciones),dtype=np.float64)
    historial_recompensas=[]
    eps= epsilon
    
    #iteramos en los episodios
    for ep in range (episodios):
        estado = env.reiniciar()
        
        #aqui iniciamos la politica epsilon grady (e-grady)
        if np.random.rand() < eps:
            accion = np.random.randint(n_acciones)
        else:
            accion = int(np.argmax(Q[estado]))
        
        siguiente_estado, recompensa, terminado, info = env.paso(accion)
        
        mejor_siguiente = np.max(Q[siguiente_estado])
        td_objetivo = recompensa + gamma * mejor_siguiente
        td_error = td_objetivo - Q[estado, accion]
        Q[estado, accion] += alpha * td_error

        historial_recompensas.append(recompensa)
        eps = max(min_epsilon, eps * decay_epsilon)
        
        if (ep + 1) % 2000 == 0:
            prom_reciente = np.mean(historial_recompensas[-5000:])
            print(f"Episodio {ep+1}/{episodios} | recompensa_promedio_ult_{2000}: {prom_reciente:.2f} | eps: {eps:.4f}")

    np.save(guardar_ruta, Q)
    print(f"Entrenamiento finalizado. Q-tabla guardada en: {guardar_ruta}")
    return Q, historial_recompensas
    

## Entrenamiento del Agente

El agente juega miles de veces.  
Usa "epsilon-greedy": explora acciones nuevas y luego aprovecha lo que ya aprendió.  
Con el tiempo comete cada vez menos errores.


In [ ]:
#aumente lo que es el reparable para que ninicie el entrenmeinto con los estaodos
Q, recompensas = entrenamiento(episodios=20000, alpha=0.2, gamma=0.99,epsilon=0.1, decay_epsilon=0.9998, min_epsilon=0.01,prob_reparable=0.2,prob_malo=0.3, semilla=123, guardar_ruta='q_tabla.npy')


Episodio 2000/20000 | recompensa_promedio_ult_2000: 6.38 | eps: 0.0670
Episodio 4000/20000 | recompensa_promedio_ult_2000: 6.54 | eps: 0.0449
Episodio 6000/20000 | recompensa_promedio_ult_2000: 6.65 | eps: 0.0301
Episodio 8000/20000 | recompensa_promedio_ult_2000: 6.75 | eps: 0.0202
Episodio 10000/20000 | recompensa_promedio_ult_2000: 6.91 | eps: 0.0135
Episodio 12000/20000 | recompensa_promedio_ult_2000: 7.01 | eps: 0.0100
Episodio 14000/20000 | recompensa_promedio_ult_2000: 7.05 | eps: 0.0100
Episodio 16000/20000 | recompensa_promedio_ult_2000: 7.00 | eps: 0.0100
Episodio 18000/20000 | recompensa_promedio_ult_2000: 7.00 | eps: 0.0100
Episodio 20000/20000 | recompensa_promedio_ult_2000: 7.03 | eps: 0.0100
Entrenamiento finalizado. Q-tabla guardada en: q_tabla.npy


## simulacion con pygame  

en esta seccion lo que hacemos es crear un funcion para poder simularlo  


In [ ]:
import pygame


#lo que hice aqui es crear otro tipo de brazo mas para poder hacer que el producto que esta para reparacion vayya a otra parte 
#como se ve tuve que modificar la mayoria de la silmulacion aumentar prob_bueno=0.5, prob_reparable=0.2, prob_malo=0.3


#como tabien agregue los colores para el reparable
def simular_pygame(q_tabla_ruta='q_tabla.npy', prob_bueno=0.5, prob_reparable=0.2, prob_malo=0.3):

    if not os.path.exists(q_tabla_ruta):
        print(f"Q-tabla no encontrada en {q_tabla_ruta}. Entrena primero.")
        return
    
    Q = np.load(q_tabla_ruta)

    pygame.init()
    ANCHO, ALTO = 640, 240
    pantalla = pygame.display.set_mode((ANCHO, ALTO))
    pygame.display.set_caption('Cinta Transportadora - 3 Estados / 3 Acciones')
    reloj = pygame.time.Clock()
    FPS = 60

   
    BLANCO = (255,255,255)
    CINTA = (120,120,120)
    BUENO = (50,180,50)
    REPARABLE = (255, 200, 50)
    MALO = (200,60,60)
    RETIRADOR = (70,130,180)
    REPARADOR = (100, 150, 255)

    
    x_retirador = ANCHO - 200
  
    x_reparador = ANCHO - 100

   
    x_producto = 60
    y_producto = ALTO//2
    velocidad = 2
    rng = random.Random(123)

    
    def nuevo_estado():
        return np.random.choice([0,1,2], p=[prob_bueno, prob_reparable, prob_malo])

    estado_producto = nuevo_estado()

    corriendo = True
    while corriendo:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False

        pantalla.fill(BLANCO)
        pygame.draw.rect(pantalla, CINTA, (0, ALTO//2 - 20, ANCHO, 40))

        estado = estado_producto
        accion = int(np.argmax(Q[estado]))

        
        if accion == 1:
            pygame.draw.rect(pantalla, RETIRADOR, (x_retirador, ALTO//2 - 60, 40, 120))
        else:
            pygame.draw.rect(pantalla, (180,180,180), (x_retirador, ALTO//2 - 60, 40, 120))

      
      #si la accion es 2 a reparar
        if accion == 2:
            pygame.draw.rect(pantalla, REPARADOR, (x_reparador, ALTO//2 - 60, 40, 120))
        else:
            pygame.draw.rect(pantalla, (180,180,180), (x_reparador, ALTO//2 - 60, 40, 120))

      
        if estado_producto == 0:
            color = BUENO
        elif estado_producto == 1:
            color = REPARABLE
        else:
            color = MALO

        pygame.draw.circle(pantalla, color, (int(x_producto), y_producto), 16)

       
        x_producto += velocidad

       
        if x_producto >= x_retirador + 10:
            if accion == 1:   
                y_producto -= 4
            elif accion == 2 and x_producto >= x_reparador:
                y_producto += 4   
            else:
                y_producto = ALTO // 2

       
        if x_producto > ANCHO + 50 or y_producto < -50 or y_producto > ALTO + 50:
            x_producto = 40
            y_producto = ALTO//2
            estado_producto = nuevo_estado()

        
        fuente = pygame.font.SysFont(None, 20)
        texto = fuente.render(
            'Acciones: 0=dejar   1=retirar   2=reparar', 
            True, (0,0,0)
        )
        pantalla.blit(texto, (10,10))

        pygame.display.flip()
        reloj.tick(FPS)

    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/guxs/anaconda3/envs/iapro/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## ejecucion de la simulacion  
al ejecutar la simulacion lo que estamos haciendo es pasar la tabla Q que tenemos almacenada

In [6]:
simular_pygame('q_tabla.npy')